In [ ]:
!pip install opencv-python

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [ ]:
import pandas as pd
import numpy as np
import tensorflow as tf
from tensorflow.keras import Sequential
from tensorflow.keras.applications.vgg16 import preprocess_input
from tensorflow.keras.applications.vgg16 import VGG16
from keras.layers import Dense
import matplotlib.pyplot as plt
import pdb
import os
import cv2
from google.colab.patches import cv2_imshow

from google.colab import drive
from google.colab import files
drive.mount("/content/drive", force_remount=True)

Mounted at /content/drive


In [ ]:
# load feature extraction model
vgg_model = VGG16(include_top=False, weights='imagenet')

# load classification model
model_fc = Sequential()
model_fc.add(Dense(1000, activation="selu"))
model_fc.add(Dense(500, activation="selu"))
model_fc.add(Dense(50, activation="selu"))
model_fc.add(Dense(1, activation="sigmoid"))
model_fc.compile(loss="binary_crossentropy", optimizer="adam")
os.chdir('/content/drive/MyDrive/CIS 5810 Final Project/Model Weights')
model_fc.load_weights('/content/drive/MyDrive/CIS 5810 Final Project/Model Weights/brad_weights')

58889256/58889256 [==============================] - 0s 0us/step


In [ ]:
# function for processing videos
def process_video(lp, sp, rp):
    vidcap = cv2.VideoCapture(lp)
    success,img = vidcap.read()
    shape = img.shape[:2] # input shape of video

    # fourcc = cv2.VideoWriter_fourcc(*'mp4v')  # use this to save .mp4 files
    fourcc = cv2.VideoWriter_fourcc(*'MJPG')  # use this to save .avi files
    fps =  vidcap.get(cv2.CAP_PROP_FPS)
    output = cv2.VideoWriter(sp, fourcc, fps, shape)

    face_cascade = cv2.CascadeClassifier(cv2.data.haarcascades + 'haarcascade_frontalface_default.xml')
    frame_number = 0
    length = int(vidcap.get(cv2.CAP_PROP_FRAME_COUNT))  # number of frames in video
    with open(rp, 'w') as f:
        while success:
            frame_number += 1
            frame_string = str(frame_number)
            gray = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)  # face dection input is a grayscale image
            faces = face_cascade.detectMultiScale(gray, 1.3, 5)  # get all faces in this frame
            coords_list = []
            probs_list = []
            for (x,y,w,h) in faces:  # tuple of bounding box coordinates (upper left corner, width, and height)
                x,y,w,h = x,y,w+20,h+20  # we expanded the width and height of bounding boxes by 20 pixels each
                face = img[y:y+h, x:x+w]  # crop image to face
                input = face.copy()
                input = cv2.resize(input, (224, 224))  # input to VGG16 needs to be 224x224 pixels
                input = np.expand_dims(input, axis=0)
                input = preprocess_input(input)
                features = vgg_model.predict(input, verbose=False)  # run image through VGG and extract features
                features = features.flatten()
                features = features.reshape(1,-1)
                class_prob = model_fc.predict(features, verbose=False).item()  # run features through linear fully connected model
                print(f'Brad Prob: {class_prob:.2%}')
                if(class_prob > 0.5):  # if we believe this faces is Brad
                    probs_list.append(class_prob)
                    coords_list.append((x,y,w,h))
                    frame_string += ',' + str(class_prob)
            try:
                most_brad_i = np.argmax(probs_list)
                most_brad_prob = np.max(probs_list)
                most_brad_coords = coords_list[most_brad_i]
                img = cv2.rectangle(img ,(x,y),(x+w,y+h),(255,0,0),2)  # draw bounding box on image
                cv2.putText(img, str(round(class_prob*100, 2))+'%', (x, y-10), cv2.FONT_HERSHEY_SIMPLEX, 0.9, (36,255,12), 2)  # add Pr[Brad] above bounding box
            except:
                pass
            
            f.write(frame_string + '\n')
            cv2.waitKey(0)
            cv2.destroyAllWindows()
            img = cv2.resize(img, shape)  # grab next frame
            output.write(img)
            print("Writing frame {} / {}".format(frame_number, length))

            success,img = vidcap.read()
            # success = False # if you only want to run for one frame

    output.release()

In [ ]:
base_path = "/content/drive/MyDrive/CIS 5810 Final Project/test_videos2/"
load_paths = [
    "brad7.mp4"
]

save_paths = [
    "brad7_output.avi"
]

results_paths = [
    "brad7_metadata.txt"
]

for i in range(len(load_paths)):
    process_video(base_path + load_paths[i], base_path + save_paths[i], base_path + results_paths[i])       

Brad Prob: 2.02%
Brad Prob: 66.84%
Writing frame 1 / 370
Brad Prob: 0.75%
Brad Prob: 78.73%
Writing frame 2 / 370
Brad Prob: 0.48%
Brad Prob: 81.23%
Writing frame 3 / 370
Brad Prob: 2.02%
Brad Prob: 85.21%
Writing frame 4 / 370
Brad Prob: 0.25%
Brad Prob: 69.85%
Writing frame 5 / 370
Brad Prob: 0.57%
Brad Prob: 71.33%
Writing frame 6 / 370
Brad Prob: 78.67%
Brad Prob: 7.86%
Writing frame 7 / 370
Brad Prob: 89.49%
Brad Prob: 0.07%
Writing frame 8 / 370
Brad Prob: 67.12%
Brad Prob: 12.09%
Writing frame 9 / 370
Brad Prob: 82.41%
Brad Prob: 0.08%
Writing frame 10 / 370
Brad Prob: 1.04%
Brad Prob: 78.02%
Writing frame 11 / 370
Brad Prob: 96.13%
Brad Prob: 7.72%
Writing frame 12 / 370
Brad Prob: 92.29%
Brad Prob: 0.37%
Writing frame 13 / 370
Brad Prob: 85.28%
Brad Prob: 0.04%
Writing frame 14 / 370
Brad Prob: 80.41%
Brad Prob: 3.11%
Writing frame 15 / 370
Brad Prob: 93.25%
Brad Prob: 0.29%
Writing frame 16 / 370
Brad Prob: 83.40%
Brad Prob: 3.91%
Writing frame 17 / 370
Brad Prob: 96.39%
Brad